In [60]:
"""
Initialization and schemas for the database
"""
import datetime

import psycopg2

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT


DB_USER = "postgres"
DB_PASS = "entropy09"
DB_HOST = "db"
DB_PORT = "5432"
DB_NAME = "planet"
DB_TABLE_NAME = "routes"

# Need to add the appropriate types here
CREATE_TABLE_SCRIPT = """
    CREATE TABLE {} (
    route_id SERIAL,
    timestamp TIMESTAMP,
    route_length REAL
    );
    """.format(
    DB_TABLE_NAME
)

ADD_POSTGIS_EXT_TO_DB_SCRIPT = "CREATE EXTENSION IF NOT EXISTS postgis"

ADD_GEOM_COLUMN_TO_TABLE_SCRIPT = (
    "SELECT AddGeometryColumn('public', 'routes', 'geom', 4326, 'POINT', 2);"
)

NEW_ROUTE_ID_SCRIPT = "SELECT max(route_id) + 1 FROM routes;"

UPDATE_ROUTE_SCRIPT = "INSERT INTO routes (route_id, timestamp, geom) VALUES ({}, now(), 'SRID=4326; POINT({} {})');"

COLLECT_POINTS_IN_ROUTE_SCRIPT = "SELECT * FROM routes WHERE route_id = {} AND geom IS NOT NULL ORDER BY timestamp;;"

COLLECT_ROUTES_IN_DAY = """SELECT * FROM routes WHERE timestamp BETWEEN '{}' and '{}'::date + interval '24 hours' AND geom IS NOT NULL;"""

def create_new_database():
    conn = psycopg2.connect(
        "dbname=postgres user={} password={}".format(DB_USER, DB_PASS)
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    cur.execute("CREATE DATABASE {};".format(DB_NAME))
    close_and_commit(cur, conn)


def activate_pg_extensions(ext_script):
    conn = psycopg2.connect(
        "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
    )
    cur = conn.cursor()
    cur.execute(ext_script)
    close_and_commit(cur, conn)


def create_table():
    conn = psycopg2.connect(
        "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
    )
    cur = conn.cursor()
    cur.execute(CREATE_TABLE_SCRIPT)
    close_and_commit(cur, conn)


def add_geometry_column():
    conn = psycopg2.connect(
        "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
    )
    cur = conn.cursor()
    cur.execute(ADD_GEOM_COLUMN_TO_TABLE_SCRIPT)
    close_and_commit(cur, conn)

# TODO: implement and test against what was supplied
# After pass test, move on to attack second question.
# Dockerfile will just be the Postgres and the Flask app communicating with one another 
# BUt, we will need to allow for external communication to the Flask app, maybe add nginx as a server?
def calculate_distance_between_points():
    # -- SELECT ST_Distance_Spheroid(geometry(a.geom), geometry(b.geom)) / 1000. as km
    # -- FROM test a, test b
    # -- WHERE a.name='Point1' AND b.name='Point2';
    pass


def create_new_route():
    """
    Gets a new route ID, and commits this to the db 
    Returns new route id for the user
    """
    conn = psycopg2.connect(
        "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
    )
    cur = conn.cursor()
    cur.execute(NEW_ROUTE_ID_SCRIPT)
    new_route_id = cur.fetchone()
    if str(new_route_id[0]) == 'None': # TODO test if we need the conversion here
        cur.execute("INSERT INTO routes (route_id, timestamp) VALUES (0, now());")
        close_and_commit(cur, conn)
        return {"route_id": 0}
    cur.execute("INSERT INTO routes (route_id, timestamp) VALUES ({}, now());".format(new_route_id[0]))
    close_and_commit(cur, conn)
    return {"route_id": str(new_route_id[0])}


def update_route(route_id, longitude, latitude):
    """
    Handles date internally
    """
    #   date = datetime.datetime.today().strftime("%d-%m-%Y")
    conn = psycopg2.connect(
        "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
    )
    cur = conn.cursor()
    cur.execute(UPDATE_ROUTE_SCRIPT.format(route_id, longitude, latitude))
    close_and_commit(cur, conn)

def close_and_commit(cur, conn):
    cur.close()
    conn.commit()
    conn.close()
    
def given_date_query_orders_correctly(date):
    conn = psycopg2.connect(
        "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
    )
    cur = conn.cursor()
    cur.execute(COLLECT_ROUTES_IN_DAY.format(date, date))
    results = cur.fetchall()
    close_and_commit(cur, conn)
    return results 

def collect_points_in_route(route_id):
    conn = psycopg2.connect(
        "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
    )
    cur = conn.cursor()
    cur.execute(COLLECT_POINTS_IN_ROUTE_SCRIPT.format(route_id))
    results = cur.fetchall()
    close_and_commit(cur, conn)
    return results 


In [2]:
create_new_database()
activate_pg_extensions(ADD_POSTGIS_EXT_TO_DB_SCRIPT)
create_table()
add_geometry_column()

In [11]:
create_new_route()

{'route_id': '2'}

In [4]:
conn = psycopg2.connect(
    "dbname={} user={} password={}".format(DB_NAME, DB_USER, DB_PASS)
)

In [5]:
cur = conn.cursor()

In [12]:
cur.execute("select * from routes;")

In [13]:
cur.fetchall()

[(0, datetime.datetime(2019, 8, 9, 19, 11, 55, 671940), None),
 (1, datetime.datetime(2019, 8, 9, 19, 12, 21, 559135), None),
 (2, datetime.datetime(2019, 8, 9, 19, 12, 32, 125507), None)]

In [20]:
update_route(2, 50.273901, 20.591889)

In [53]:
cur.execute("select * from routes;")

In [54]:
cur.fetchall()

[(0, datetime.datetime(2019, 8, 9, 19, 11, 55, 671940), None),
 (1, datetime.datetime(2019, 8, 9, 19, 12, 21, 559135), None),
 (2, datetime.datetime(2019, 8, 9, 19, 12, 32, 125507), None),
 (2,
  datetime.datetime(2019, 8, 9, 19, 13, 9, 387273),
  '0101000020E6100000BDAB1E300F234A40B5DC990986973240'),
 (2,
  datetime.datetime(2019, 8, 9, 19, 14, 7, 764402),
  '0101000020E6100000BDAB1E300FA34940B5DC990986973340'),
 (2,
  datetime.datetime(2019, 8, 9, 19, 14, 14, 796710),
  '0101000020E6100000BDAB1E300F234940B5DC990986973440'),
 (5, datetime.datetime(2019, 11, 11, 19, 11, 55, 156537), None)]

In [38]:
cur.execute(COLLECT_POINTS_IN_ROUTE_SCRIPT.format(2))

In [39]:
route_2 = cur.fetchall()

In [62]:
route_2

[(2,
  datetime.datetime(2019, 8, 9, 19, 13, 9, 387273),
  '0101000020E6100000BDAB1E300F234A40B5DC990986973240'),
 (2,
  datetime.datetime(2019, 8, 9, 19, 14, 7, 764402),
  '0101000020E6100000BDAB1E300FA34940B5DC990986973340'),
 (2,
  datetime.datetime(2019, 8, 9, 19, 14, 14, 796710),
  '0101000020E6100000BDAB1E300F234940B5DC990986973440')]

In [42]:
[i[2] for i in route_2]

['0101000020E6100000BDAB1E300F234A40B5DC990986973240',
 '0101000020E6100000BDAB1E300FA34940B5DC990986973340',
 '0101000020E6100000BDAB1E300F234940B5DC990986973440']

In [61]:
given_date_query_orders_correctly("2019-08-09")

[(2,
  datetime.datetime(2019, 8, 9, 19, 13, 9, 387273),
  '0101000020E6100000BDAB1E300F234A40B5DC990986973240'),
 (2,
  datetime.datetime(2019, 8, 9, 19, 14, 7, 764402),
  '0101000020E6100000BDAB1E300FA34940B5DC990986973340'),
 (2,
  datetime.datetime(2019, 8, 9, 19, 14, 14, 796710),
  '0101000020E6100000BDAB1E300F234940B5DC990986973440')]

In [ ]:
# -- select * from routes;

# -- insert into routes (date, geom) values ('now'::abstime, 'SRID=4326; POINT(-25.4025905 -49.3124416)');

# -- select * from routes;

# insert into routes (route_id, date, geom) values (1, 'now'::abstime, 'SRID=4326; POINT(-25.4025905 -49.3124416)')

In [ ]:
# -- alter table test add column geog geography;
# -- insert into your_table (geog) values ('SRID=4326;POINT(longitude latitude)');
# -- insert into test (geog) values ('SRID=4326;POINT(-25.4025905 -49.3124416)');

# -- select * from test;

# -- insert into test (name, amount, geog) values ('Esra', 10, ('SRID=4326;POINT(-25.4025905 -49.3124416)'))

# -- select geog from test where name like 'Esra';
# -- {"lat": -25.4025905, "lon": -49.3124416},
# -- {"lat": -23.559798, "lon": -46.634971},
# -- {"lat": 59.3258414, "lon": 17.70188},
# -- {"lat": 54.273901, "lon": 18.591889},

# -- insert into test (name, amount, geog) values ('Point1', 10, ('SRID=4326;POINT(-25.4025905 -49.3124416)'));
# -- insert into test (name, amount, geog) values ('Point2', 10, ('SRID=4326;POINT(-23.559798 -46.634971)'));
# -- insert into test (name, amount, geog) values ('Point3', 10, ('SRID=4326;POINT(59.3258414 17.70188)'));
# -- insert into test (name, amount, geog) values ('Point4', 10, ('SRID=4326;POINT(54.273901 18.591889)'));

# -- select * from test;

# -- SELECT ST_Distance_Spheroid(geometry(a.geog), geometry(b.geog)) / 1000. as km
# -- FROM test a, test b
# -- WHERE a.name='Point1' AND b.name='Point2';

# -- SELECT ST_Distance_Spheroid(geometry(a.geog), geometry(b.geog)) / 1000. as km
# -- FROM test a, test b
# -- WHERE a.name='Point2' AND b.name='Point3';

# SELECT ST_Distance_Sphere(geometry(a.geog), geometry(b.geog)) / 1000. as km
# FROM test a, test b
# WHERE a.name='Point3' AND b.name='Point4';

In [64]:
datetime.datetime.now().strftime("%d-%m-%Y")

'09-08-2019'

In [2]:
334.83 + 10927.08 + 555.59

11817.5

In [ ]:
11814.6